In [2]:
# Librerias

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go

In [3]:
# Constantes

umbral_pico = 0.5

In [4]:
# Abrir los graficos en el navegador

pio.renderers.default = 'browser'

In [5]:
# Carga de datos

df_ple7= pd.read_csv(r"../data/9-1-2026/data-1768216236753.csv")
df_ple1 = pd.read_csv(r"../data/9-1-2026/data-1768216350513.csv")
# df_data_raw = pd.read_csv(r"../data/data-raw30122025.csv", encoding='latin1')

df_ple1['maquina'] = 'Plegadora 1'
df_ple7['maquina'] = 'Plegadora 7'

# Ordenamiento de los datos en base a su marca temporal
df_ple1 = df_ple1.sort_values('temporal_placa').reset_index(drop=True)  
df_ple7 = df_ple7.sort_values('temporal_placa').reset_index(drop=True)


In [6]:
# Funcion preparar_df
# Esta funcion toma los datos de la plegadora y asigna a cada fila el turno correspondiente 

def preparar_df(df):
    df = df.copy()

    # Timestamp
    df['temporal_placa'] = pd.to_datetime(df['temporal_placa'])
    df['hora'] = df['temporal_placa'].dt.hour
    df['minuto'] = df['temporal_placa'].dt.minute

    # Turnos
    def asignar_turno(hora, minuto):
        t = hora * 60 + minuto

        # Pausas
        if 12*60 <= t < 12*60 + 30:
            return 'ALMUERZO'
        if 22*60 <= t < 22*60 + 30:
            return 'CENA'

        # Turnos
        if 5*60 <= t < 17*60:
            return 'TURNO MAÑANA'
        if 17*60 <= t < 22*60:
            return 'TURNO TARDE'
        if (t >= 22*60 + 30) or (t < 1*60):
            return 'TURNO TARDE'

        return 'FUERA_TURNO'

    df['turno'] = df.apply(
        lambda x: asignar_turno(x['hora'], x['minuto']),
        axis=1
    )
    
    # Potencias totales por timestamp
    df['p_activa_total'] = (
        df['potencia_a_r'] +
        df['potencia_a_s'] +
        df['potencia_a_t']
    )

    df['q_reactiva_total'] = (
        df['potencia_r_r'] +
        df['potencia_r_s'] +
        df['potencia_r_t']
    )

    return df


In [7]:
# Ajuste datos de las plegadoras

df_ple1 = preparar_df(df_ple1)
df_ple7 = preparar_df(df_ple7)

df_all = pd.concat([df_ple1, df_ple7], ignore_index=True)

df_all = df_all.sort_values(['maquina','temporal_placa']).reset_index(drop=True)

# Filtra los datos correspondientes al 10-1-2026
import datetime as dt

dia = dt.date(2026, 1, 9)
df_all = df_all[df_all['temporal_placa'].dt.date == dia]

In [8]:
# Funcion detectar_picos
def detectar_picos(signal, timestamp):
    """
    signal: pd.Series
    timestamp
    Devuelve índices de picos
    """
    dt = timestamp.diff().dt.total_seconds()
    dI_dt = signal.diff() / dt

    threshold = umbral_pico

    return signal[
        (dI_dt > threshold)
    ].index


In [9]:
# Deteccion de picos

import pandas as pd

maquinas = df_all['maquina'].unique()

medias_por_fase = (
    df_all
    .groupby('maquina')
    .agg(
        I_R_media=('corriente_r', 'mean'),
        I_S_media=('corriente_s', 'mean'),
        I_T_media=('corriente_t', 'mean')
    )
)

medias_por_fase['fase_mas_cargada'] = medias_por_fase[
    ['I_R_media', 'I_S_media', 'I_T_media']
].idxmax(axis=1)

fase_col = {
    'I_R_media': 'corriente_r',
    'I_S_media': 'corriente_s',
    'I_T_media': 'corriente_t'
}

fase_label = {
    'I_R_media': 'Fase R',
    'I_S_media': 'Fase S',
    'I_T_media': 'Fase T'
}

# Deteccion
df_peaks_list = []

for maq in maquinas:

    fase_media = medias_por_fase.loc[maq, 'fase_mas_cargada']
    col_corriente = fase_col[fase_media]

    g = (
        df_all[df_all['maquina'] == maq]
        .sort_values('temporal_placa')
        .copy()
    )

    señal = g[col_corriente]
    timestamp = g['temporal_placa']

    idx_picos = detectar_picos(señal, timestamp)

    df_peaks = g.loc[idx_picos, ['temporal_placa']].copy()
    df_peaks['maquina'] = maq
    df_peaks['fase'] = fase_label[fase_media]

    df_peaks_list.append(df_peaks)


# Nuevo dataframe
df_peaks_all = pd.concat(df_peaks_list, ignore_index=True)

df_peaks_all['temporal_placa'] = (
    pd.to_datetime(df_peaks_all['temporal_placa'], errors='coerce')
    .dt.tz_localize(None)
)


In [10]:
# Visualizacion de picos detectados y su marca temporal

df_peaks_all = df_peaks_all.sort_values(
    by=['maquina', 'temporal_placa']
).reset_index(drop=True)

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

#df_peaks_all

In [24]:
# Analisis de Pasos
import pandas as pd

SILENCIO_MAX = pd.Timedelta(minutes=0.7)

rows = []

for maq, g in df_peaks_all.groupby('maquina'):

    g = g.sort_values('temporal_placa').reset_index(drop=True)

    # diferencias con vecino anterior y siguiente
    g['dt_prev'] = g['temporal_placa'] - g['temporal_placa'].shift(1)
    g['dt_next'] = g['temporal_placa'].shift(-1) - g['temporal_placa']

    # pico espurio: aislado por ambos lados
    g['pico_espurio'] = (
        (g['dt_prev'] > 3 * SILENCIO_MAX) &
        (g['dt_next'] > 3 * SILENCIO_MAX)
    )

    print(f'\n{maq} – picos espurios detectados:')
    print(g.loc[g['pico_espurio'],
                ['temporal_placa', 'dt_prev', 'dt_next']])

    # filtrar picos espurios
    g_filtrado = g[~g['pico_espurio']].copy()

    if g_filtrado.empty:
        print(f'⚠ {maq}: todos los picos fueron filtrados')
        continue

    # diferencia entre picos consecutivos (CORRECTO)
    g_filtrado['dt'] = g_filtrado['temporal_placa'].diff()

    # nuevo paso
    g_filtrado['nuevo_paso'] = (
        g_filtrado['dt'].isna() |
        (g_filtrado['dt'] > SILENCIO_MAX)
    )

    g_filtrado['paso_id'] = g_filtrado['nuevo_paso'].cumsum()

    # agrupar por paso
    for paso_id, c in g_filtrado.groupby('paso_id'):

        fecha_inicio = c['temporal_placa'].iloc[0]
        fecha_fin    = c['temporal_placa'].iloc[-1]

        rows.append({
            'maquina': maq,
            'paso_id': paso_id,
            'fecha_inicio': fecha_inicio,
            'fecha_fin': fecha_fin,
            'duracion': fecha_fin - fecha_inicio
        })

df_pasos = pd.DataFrame(rows)


pd.set_option('display.max_rows', None) 
pd.set_option('display.max_columns', None) 
pd.set_option('display.width', None) 

df_pasos



Plegadora 1 – picos espurios detectados:
         temporal_placa         dt_prev         dt_next
209 2026-01-09 21:34:45 0 days 00:17:33 0 days 00:13:05

Plegadora 7 – picos espurios detectados:
         temporal_placa         dt_prev         dt_next
13  2026-01-09 17:40:45 0 days 00:03:14 0 days 00:11:22
86  2026-01-09 20:20:06 0 days 00:02:42 0 days 00:12:13
134 2026-01-09 21:03:58 0 days 00:03:30 0 days 00:02:54
142 2026-01-09 21:25:49 0 days 00:09:36 0 days 00:15:20
165 2026-01-09 22:04:41 0 days 00:09:16 0 days 01:02:58


,maquina,paso_id,fecha_inicio,fecha_fin,duracion
0,Plegadora 1,1,2026-01-09 16:57:16,2026-01-09 16:58:04,0 days 00:00:48
1,Plegadora 1,2,2026-01-09 17:04:53,2026-01-09 17:05:41,0 days 00:00:48
2,Plegadora 1,3,2026-01-09 17:11:22,2026-01-09 17:11:57,0 days 00:00:35
3,Plegadora 1,4,2026-01-09 17:15:09,2026-01-09 17:16:05,0 days 00:00:56
4,Plegadora 1,5,2026-01-09 17:17:43,2026-01-09 17:18:54,0 days 00:01:11
5,Plegadora 1,6,2026-01-09 17:20:12,2026-01-09 17:20:40,0 days 00:00:28
6,Plegadora 1,7,2026-01-09 18:07:26,2026-01-09 18:07:38,0 days 00:00:12
7,Plegadora 1,8,2026-01-09 18:09:52,2026-01-09 18:10:05,0 days 00:00:13
8,Plegadora 1,9,2026-01-09 18:13:54,2026-01-09 18:14:17,0 days 00:00:23
9,Plegadora 1,10,2026-01-09 18:15:03,2026-01-09 18:15:20,0 days 00:00:17


In [29]:
# Analisis de Ciclos
import pandas as pd

rows = []
# Parámetro clave
SILENCIO_MAX_CICLO = pd.Timedelta(minutes=20)

for maq, g in df_pasos.groupby('maquina'):

    g = g.sort_values('fecha_inicio').reset_index(drop=True)
    # diferencia entre pasos
    g['dif_temp'] = g['fecha_inicio']-g['fecha_fin'].shift(1)

    # nuevo ciclo si:
    # - hay diferencia de tiempo entre el final de un paso y el comienzo del siguiente de al menos 25 minutos
    g['nuevo_ciclo'] = (g['dif_temp'].isna()) | (g['dif_temp'] > SILENCIO_MAX_CICLO)

    # id de paso incremental
    g['ciclo_id'] = g['nuevo_ciclo'].cumsum()

    # agrupar por ciclo
    for ciclo_id, c in g.groupby('ciclo_id'):

        comienzo_ciclo = c['fecha_inicio'].iloc[0]
        fin_ciclo   = c['fecha_fin'].iloc[-1]

        picos_en_ciclo = df_peaks_all[
        (df_peaks_all['maquina'] == maq) &
        (df_peaks_all['temporal_placa'] >= comienzo_ciclo) &
        (df_peaks_all['temporal_placa'] <= fin_ciclo)
    ]
        golpes_ciclo = len(picos_en_ciclo) -1

        rows.append({
            'Maquina': maq,
            'ID_Ciclo': ciclo_id,
            'golpes_ciclo': golpes_ciclo,
            'Comienzo_Ciclo': comienzo_ciclo,
            'Fin_Ciclo': fin_ciclo,
            'duracion_ciclo': fin_ciclo - comienzo_ciclo
        })

df_ciclos = pd.DataFrame(rows)

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

df_ciclos

,Maquina,ID_Ciclo,golpes_ciclo,Comienzo_Ciclo,Fin_Ciclo,duracion_ciclo
0,Plegadora 1,1,25,2026-01-09 16:57:16,2026-01-09 17:20:40,0 days 00:23:24
1,Plegadora 1,2,71,2026-01-09 18:07:26,2026-01-09 19:02:23,0 days 00:54:57
2,Plegadora 1,3,55,2026-01-09 19:32:34,2026-01-09 20:09:49,0 days 00:37:15
3,Plegadora 1,4,54,2026-01-09 20:31:45,2026-01-09 21:17:12,0 days 00:45:27
4,Plegadora 1,5,13,2026-01-09 21:47:50,2026-01-09 21:53:46,0 days 00:05:56
5,Plegadora 1,6,45,2026-01-09 23:09:31,2026-01-09 23:41:17,0 days 00:31:46
6,Plegadora 7,1,37,2026-01-09 17:16:18,2026-01-09 18:14:08,0 days 00:57:50
7,Plegadora 7,2,14,2026-01-09 19:07:39,2026-01-09 19:13:12,0 days 00:05:33
8,Plegadora 7,3,88,2026-01-09 19:44:14,2026-01-09 21:16:13,0 days 01:31:59
9,Plegadora 7,4,21,2026-01-09 21:41:09,2026-01-09 21:55:25,0 days 00:14:16


In [27]:
# Visualizacion corrientes + Ciclos de trabajo

import plotly.graph_objects as go
import pandas as pd

# Determinar fase más cargada por MEDIA (por máquina)
medias_por_fase = (
    df_all
    .groupby('maquina')
    .agg(
        I_R_media=('corriente_r', 'mean'),
        I_S_media=('corriente_s', 'mean'),
        I_T_media=('corriente_t', 'mean')
    )
)

medias_por_fase['fase_mas_cargada'] = medias_por_fase[
    ['I_R_media', 'I_S_media', 'I_T_media']
].idxmax(axis=1)

map_fase_col = {
    'I_R_media': 'corriente_r',
    'I_S_media': 'corriente_s',
    'I_T_media': 'corriente_t'
}

map_fase_label = {
    'I_R_media': 'Fase R',
    'I_S_media': 'Fase S',
    'I_T_media': 'Fase T'
}

# Figuras por máquina
figs = {}
maquinas = medias_por_fase.index.tolist()

for maq in maquinas:

    fase_media = medias_por_fase.loc[maq, 'fase_mas_cargada']
    fase_col   = map_fase_col[fase_media]
    fase_label = map_fase_label[fase_media]

    g_elec = df_all[df_all['maquina'] == maq]
    g_ciclos = df_ciclos[df_ciclos['Maquina'] == maq]

    fig = go.Figure()

    # Señal eléctrica (fase más cargada)
    fig.add_trace(
        go.Scatter(
            x=g_elec['temporal_placa'],
            y=g_elec[fase_col],
            mode='lines',
            line=dict(color='#1f77b4', width=2),
            name=f'{maq} – {fase_label}',
            hovertemplate=(
                'Tiempo: %{x}<br>'
                'Corriente RMS: %{y:.1f} A<br>'
                f'{fase_label}<extra></extra>'
            )
        )
    )

   
    # Sombreado de ciclos
    shapes = []

    for _, row in g_ciclos.iterrows():
        shapes.append(
            dict(
                type='rect',
                xref='x',
                yref='paper',
                x0=row['Comienzo_Ciclo'],
                x1=row['Fin_Ciclo'],
                y0=0,
                y1=1,
                fillcolor='rgba(0, 0, 0, 0.10)',
                line=dict(width=0),
                layer='below'
            )
        )

    fig.update_layout(shapes=shapes)

    # Layout
    fig.update_layout(
        title=dict(
            text=f'{maq} – {fase_label} (fase más cargada + ciclos)',
            x=0.5,
            xanchor='center',
            font=dict(size=18)
        ),
        xaxis_title='Tiempo',
        yaxis_title='Corriente RMS [A]',
        showlegend=False
    )

    figs[maq] = fig


# Figura master con dropdown
fig_master = go.Figure()

for i, maq in enumerate(maquinas):
    for tr in figs[maq].data:
        tr.visible = (i == 0)
        fig_master.add_trace(tr)

fig_master.layout.shapes = figs[maquinas[0]].layout.shapes

# Dropdown
buttons = []
offset = 0

for i, maq in enumerate(maquinas):
    n_traces = len(figs[maq].data)

    visible = [False] * len(fig_master.data)
    for j in range(offset, offset + n_traces):
        visible[j] = True

    buttons.append(
        dict(
            label=maq,
            method='update',
            args=[
                {'visible': visible},
                {
                    'title.text': figs[maq].layout.title.text,
                    'shapes': figs[maq].layout.shapes
                }
            ]
        )
    )

    offset += n_traces

fig_master.update_layout(
    updatemenus=[
        dict(
            buttons=buttons,
            direction='down',
            showactive=True,
            x=1.02,
            y=1.0,
            xanchor='left',
            yanchor='top'
        )
    ],
    margin=dict(r=260)
)

fig_master.show()
